# A Developer's Guide to Model Onboarding in AIOS

Welcome to this guide on onboarding a model to the OpenOS/AIGr.id platform. This notebook serves as a standalone tutorial that walks you through every step of the process, from registering your model as a digital asset to running inference and managing its lifecycle. We will use the `gemma3_llama_cpp` model as our primary example.

## The AIOS Ecosystem: A Brief Overview
- [AIOS Ecosystem Architecture](https://docs.aigr.id/assets/aios-all-arch.drawio.png)
Before we dive in, it's helpful to understand the key components of the AIOS ecosystem. AIOS is designed as a decentralized, modular, and extensible platform for AI development and deployment. Its architecture consists of several core services that work together to manage the lifecycle of AI models and applications. These include:

- **Asset DB Registry**: A central catalog for discovering and managing versioned, runnable software components (Assets).
- **Resource Allocator**: Responsible for scheduling and allocating resources for running Assets on the network of clusters.
- **Cluter Controller**: Manages the lifecycle of a Block, which is a running instance of an Asset.
- **Metrics System**: A comprehensive system for monitoring the health, status, and performance of Blocks.

This guide will touch on each of these components as we walk through the model onboarding process.

## Onboarding Process Overview

In this guide, we will cover the following steps in detail:

1. **Registering an Asset**: We will create a `component.json` file that describes our model and register it with the AIOS Component Registry.
2. **Allocating a Block**: We will define the block's configuration in an `allocation.json` file and allocate a block using the AIOS API.
3. **Checking Block Status & Metrics**: We will use `curl` commands to check the block's status, health, and metrics.
4. **Performing Inference**: We will write a Python script to send an inference request to the block via gRPC.
5. **Chat UI with streaming**: We will launch an interactive chat interface to demonstrate streaming capabilities.
6. **Cleaning Up**: We will deallocate the block and deregister the asset when done.


Let's get started!


The following image provides a high-level overview of the entire model onboarding process, from defining the asset to monitoring the running service.

<!-- ![Model Onboarding Process](block_onboarding.gif) -->
<img src="onboarding.png" alt="Custom Autoscaler" width="1000" height="1000">

### A note on using prebuilt docker images. 
Below docker images can be a quick start for testing the features of AIOS. For custom model onboarding refer 
    - [Custom Model Onboarding](https://github.com/OpenCyberspace/OpenOS.AI-Documentation/blob/main/llm-docs/llm-method-1.md)
- `MANAGEMENTMASTER:31280/llama4-scout-17b:v1`
- `MANAGEMENTMASTER:31280/gemma3-27b:v1`
- `MANAGEMENTMASTER:31280/deepseek-r1-distill-70b:v1`
- `MANAGEMENTMASTER:31280/magistral-small-2506-llama-cpp:v1`
- `MANAGEMENTMASTER:31280/qwen-3-32b-llama-cpp:v1`


## 1. Registering an Asset

First, we need to define our model as an **Asset**. An asset is a static, versioned, and runnable software component that is registered in the AIOS **Asset DB Registry**. The registry acts as a central catalog, allowing developers to discover, share, and reuse assets across the ecosystem. By registering an asset, you are making it available for deployment on the AIOS network.

For a deeper dive into how the asset registry works, you can refer to the official documentation:
- [Asset DB Registry Concepts](https://github.com/OpenCyberspace/OpenOS.AI-Documentation/blob/main/assets-db-registry/assets-db-registry.md)

Let's look at the `component.json` file for `gemma3_llama_cpp`. This file is the asset's manifest, containing all the essential information AIOS needs. It defines:
- **`componentId`**: The unique name, version, and release tag for the asset.
- **`componentType`**: Specifies that this is a `model` asset.
- **`containerRegistryInfo`**: Points to the container image (`gemma3-27b:v1`) and includes metadata like the author and a description.
- **`componentMetadata`**: A rich set of details including the model's use case (`chat-completion, multi-modal`), hardware requirements (`gpu`), performance benchmarks (`MMLU`, `HellaSwag`), and known limitations.
- **`componentInitData`**: Specifies the default model files to be loaded, such as the GGUF file for the model (`gemma-3-27b-it-UD-Q8_K_XL.gguf`) and the CLIP model for multi-modal capabilities.

This file effectively serves as a comprehensive "passport" for the model within the AIOS ecosystem.

In [1]:
!cat component.json 

{
  "componentId": {
    "name": "gemma3-27b",
    "version": "1.0.0",
    "releaseTag": "stable"
  },
  "componentType": "model",
  "containerRegistryInfo": {
    "containerImage": "MANAGEMENTMASTER:31280/gemma3-27b:v1",
    "containerRegistryId": "MANAGEMENTMASTER:31280/gemma3-27b:v1",
    "containerImageMetadata": {
      "author": "llm-team",
      "description": "AIOS block for chat and multi-modal inference using llama-cpp-python with Gemma-3-27B and CLIP support"
    },
    "componentMode": "aios"
  },
  "componentMetadata": {
    "usecase": "chat-completion, multi-modal",
    "framework": "llama-cpp-python",
    "hardware": "gpu",
    "supports_local_models": true,
    "model_path_resolution": "automatic",
    "supports_quantization": true,
    "quantization_methods": [
      "4bit",
      "8bit",
      "fp16",
      "fp8"
    ],
    "provider": {
      "name": "AIOS-Internal",
      "modelIdentifier": "gemma3-27b-v1"
    },
    "architecture": {
      "contextLength": 8192,
  

In [4]:
# Register the asset with the AIOS Component Registry
!curl -X POST http://MANAGEMENTMASTER:30112/api/registerComponent -H "Content-Type: application/json" -d @./component.json | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9654  100    91  100  9563  13622  1398k --:--:-- --:--:-- --:--:-- 1571k
{
   "error" : true,
   "message" : "Component with URI model.gemma3-27b:1.0.0-stable already exists."
}


## 2. Allocating a Block

Now that the asset is registered, we can create a running instance of it. In AIOS, a running instance of an asset is called a **Block**. A Block is the fundamental unit of execution in AIOS, responsible for serving inference requests or running any computational workload defined by an Asset. When you allocate a Block, the AIOS **Resource Allocator** finds a suitable cluster and schedules the Block for execution.

For more details, you can refer to the official documentation:
- [Block Concepts](https://github.com/OpenCyberspace/OpenOS.AI-Documentation/blob/main/block/block.md)
- [Block APIs/Services](https://github.com/OpenCyberspace/OpenOS.AI-Documentation/blob/main/block/block.md#block-services)


To allocate a block, we must provide an `allocation.json` file. This file is a request to the Resource Allocator, specifying the desired configuration,specification for the block. Let's examine the key fields in our `allocation.json` for the `gemma3_llama_cpp` model. This file tells AIOS:

- **`blockId`**: We are requesting a block named `gemma3-27b-block`.
- **`blockComponentURI`**: The block should be an instance of the `model.gemma3-27b:1.0.0-stable` asset we registered earlier.
- **`blockInitData`**: This section provides initial data to the block. Here we specify the `model_name` (`gemma-3-27b-it-UD-Q8_K_XL.gguf`) and the `clip_model_name` (`mmproj-F16.gguf`) for multi-modal capabilities.
- **`initSettings`**: These are settings for the block's runtime. We have settings for `tensor_parallel`, `device` (`cuda`), `quantization_type` (`fp16`), and `gen_params` which control the model's generation behavior (e.g., `max_new_tokens`, `temperature`).
- **`policyRulesSpec`**: This is a crucial section that defines the chain of policies governing the block's behavior:
    - **`accessChecker`**: Like a policing system which is embedded in `AIOS ecosystem` ,can be restricted zones ,region related polcies
    - **`clusterAllocator`**: Selects a specific cluster, in this case, `gcp-cluster-2`.
    - **`resourceAllocator`**: Assigns the block to a specific node (`wc-gpu-node1`) and GPU (`0`).
    - **`loadBalancer`**: Manages how requests are distributed, configured here to cache sessions.
    - **`stabilityChecker`**: Defines a health check mechanism to ensure the block is running correctly.
    - **`autoscaler`**: Enables autoscaling for the block, an example could be targeting token utilization.


In [5]:
!cat allocation.json

{
    "head": {
        "templateUri": "Parser/V1",
        "parameters": {}
    },
    "body": {
        "spec": {
            "values": {
                "mode": "allocate",
                "blockId": "gemma3-27b-block2",
                "blockComponentURI": "model.gemma3-27b:1.0.0-stable",
                "minInstances": 1,
                "maxInstances": 3,
                "blockInitData": {
                    "model_name": "gemma-3-27b-it-UD-Q8_K_XL/gemma-3-27b-it-UD-Q8_K_XL.gguf",
                    "clip_model_name": "gemma-3-27b-it-UD-Q8_K_XL/mmproj-F16.gguf"
                },
                "initSettings": {
                    "tensor_parallel": true,
                    "device": "cuda",
                    "quantization_type": "fp16",
                    "cleanup_enabled":  true,
                    "cleanup_check_interval": 60,
                    "cleanup_session_timeout": 120,
                    "gen_params": {
                        "max_new_tokens": 2048,
       

In [6]:
#Now, let's allocate the block using `curl`.
!curl -X POST -d @./allocation.json -H "Content-Type: application/json" http://MANAGEMENTMASTER:30501/api/createBlock

{
  "result": {
    "data": {
      "message": "task scheduled in background",
      "task_id": "67bd2c5c-d2a0-4498-bf33-93d3f2d3f675"
    },
    "success": true
  },
  "success": true,
  "task_id": ""
}


## 3. Checking Block Status and Metrics

After allocating the block, it's important to check its status to ensure it's running correctly. AIOS provides a comprehensive **Metrics System** that exposes endpoints for health status, and resource consumption, giving you a complete picture of your block's operational state. The Metrics System is designed to be extensible, allowing you to define and expose custom metrics for your applications.

For more information on the metrics system, see the documentation:
- [Metrics System Overview-covered in later tutorial](https://github.com/OpenCyberspace/OpenOS.AI-Documentation/blob/main/metrics-system/metrics-system.md#metrics-system)
- [Custom Metrics-covered in later tutorial](https://github.com/OpenCyberspace/OpenOS.AI-Documentation/blob/main/metrics-system/metrics-system.md#custom-metrics)

Let's check the health status and metrics of our block.

### Check Block Health


In [7]:
# Check Block Health - This command verifies the health of the running service within the block.
!curl -X GET http://MANAGEMENTMASTER:30201/block/health/gemma3-27b-block2

{
  "block_id": "gemma3-27b-block2",
  "healthy": true,
  "instances": [
    {
      "healthy": true,
      "instanceId": "executor",
      "reason": "executor instance"
    },
    {
      "healthy": true,
      "instanceId": "in-0kcz",
      "lastMetrics": "22.805049657821655s ago"
    }
  ],
  "success": true
}


### Check Block Metrics(Before Inference)

#### Core Metric Categories

##### 🔧 **Runtime Metrics**
- **CPU Usage**: Real-time CPU utilization percentage for the block container
- **Memory Usage**: Current memory consumption and peak memory usage
- **GPU Utilization**: GPU usage percentage and VRAM consumption (for GPU-enabled blocks)

##### 📊 **Performance Metrics**
- **Request Latency**: Average,  response times for inference requests
- **Throughput**: Requests per second (RPS) and tokens per second (TPS)
- **Queue Length**: Number of pending requests in the processing queue

##### 🔄 **Operational Metrics**
- **Request Count**: Total number of requests processed over time
- **Error Rate**: Percentage of failed requests and error types

##### 🏥 **Health Metrics**
- **Uptime**: Total running time since last restart


In [8]:
# Check Block Metrics before launching the app
!curl -X GET http://MANAGEMENTMASTER:30201/block/gemma3-27b-block2 | json_pp #show case the metrics we are getting,hardware,online, our custom 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3105  100  3105    0     0   101k      0 --:--:-- --:--:-- --:--:--  101k
{
   "data" : [
      {
         "blockId" : "gemma3-27b-block2",
         "instances" : [
            {
               "blockId" : "gemma3-27b-block2",
               "instanceId" : "executor",
               "latency" : {
                  "latency" : 0
               },
               "nodeKey" : "executor__gemma3-27b-block2",
               "tasks_processed" : {
                  "tasks_processed_created" : 1754029886.04923,
                  "tasks_processed_total" : 0
               },
               "type" : "app",
               "uptime" : 610.118766069412,
               "uptime_hours" : 0.169477435019281,
               "uptime_minutes" : 10.1686461011569
            },
            {
               "blockId" : "gemma3-27b-block2",
            

## 4. Performing Inference

Now that the block is running, let's perform an inference task. We'll use a Python script to send a request to the block via gRPC.


In [ ]:
# First, let's install the necessary libraries for our gRPC client and import them.
# We also need to add the `inference_client` directory to our Python path to import the generated gRPC files.
!pip install grpcio grpcio-tools protobuf

import sys
sys.path.append('../utils/inference_client')

import grpc
import json
import time

import service_pb2
import service_pb2_grpc

Now, let's define a function to send an inference request to our block. This function will connect to the gRPC server, construct a request packet, and print the response.

In [10]:
def run_inference(block_id, session_id, seq_no, message, generation_config, image_url=None):
    SERVER_ADDRESS = "CLUSTER1MASTER:31500"
    
    # Connect to the gRPC server
    channel = grpc.insecure_channel(SERVER_ADDRESS)
    stub = service_pb2_grpc.BlockInferenceServiceStub(channel)

    if image_url:
        data = {
            "mode": "chat",
            "gen_params": generation_config,
            "messages": [{
                "content": [
                    {"type": "text", "text": message},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]
            }]
        }
    else:
        data = {
            "mode": "chat",
            "message": message,
            "gen_params": generation_config
        }

    # Create the BlockInferencePacket request
    request = service_pb2.BlockInferencePacket(
        block_id=block_id,
        session_id=session_id,
        seq_no=seq_no,
        data=json.dumps(data),
        ts=time.time()
    )

    try:
        st = time.time()
        # Make the gRPC call
        response = stub.infer(request)
        et = time.time()

        print(f"Latency: {et - st}s")
        print(f"Session ID: {response.session_id}")
        print(f"Sequence No: {response.seq_no}")
        
        # Parse JSON response data
        try:
            response_data = json.loads(response.data)
            print("Data:")
            print(json.dumps(response_data, indent=2))
        except (json.JSONDecodeError, TypeError):
             print(f"Data: {response.data}")

        print(f"Timestamp: {response.ts}")

    except grpc.RpcError as e:
        print(f"gRPC Error: {e.code()} - {e.details()}")

In [12]:
generation_config = {
    "temperature": 0.1,
    "top_p": 0.95,
    "max_tokens": 512
}

run_inference(
    block_id="gemma3-27b-block2",
    session_id="session_notebook_multimodal-3",
    seq_no=1,
    message="Analyze the following image and generate your objective scene report.Suggest is there any snatching like activity",
    generation_config=generation_config,
    image_url="https://akm-img-a-in.tosshub.com/indiatoday/images/story/202311/chain-snatching-caught-on-camera-in-bengaluru-293151697-16x9_0.jpg"
)

Latency: 16.481139421463013s
Session ID: session_notebook_multimodal-3
Sequence No: 1
Data:
{
  "reply": "Here's an objective scene report based on the provided images:\n\n**Scene Description:**\n\nThe images depict a street scene with several individuals and vehicles. The road appears wet, possibly due to rain. \n\n*   **People:** There are at least four people visible. A woman in a green and red outfit is walking with a scooter. A man in a dark blue shirt is walking alongside her. Another person is visible near an auto-rickshaw (tuk-tuk). A fourth person is lying on the ground.\n*   **Vehicles:** An auto-rickshaw is present. Several cars are visible in the background.\n*   **Sequence:** The images appear to be a sequence, showing a progression of events.\n\n**Possible Snatching Activity:**\n\nBased on the sequence of images, it *appears* there may be a snatching or assault taking place. Here's why:\n\n*   **Sudden Fall:** The woman with the scooter and the man walking with her are up

## 4.a Block Metrics After Inference

In [15]:
# Check Block Metrics before launching the app
!curl -X GET http://MANAGEMENTMASTER:30201/block/gemma3-27b-block2 | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5776  100  5776    0     0   243k      0 --:--:-- --:--:-- --:--:--  245k
{
   "data" : [
      {
         "blockId" : "gemma3-27b-block2",
         "instances" : [
            {
               "blockId" : "gemma3-27b-block2",
               "instanceId" : "executor",
               "latency" : {
                  "latency" : 0.000175237655639648
               },
               "nodeKey" : "executor__gemma3-27b-block2",
               "tasks_processed" : {
                  "tasks_processed_created" : 1754029886.04923,
                  "tasks_processed_total" : 2
               },
               "type" : "app",
               "uptime" : 1330.24674320221,
               "uptime_hours" : 0.369512984222836,
               "uptime_minutes" : 22.1707790533702
            },
            {
               "blockId" : "gemma3-27b-bl

## 5. Interactive Chat with Streamlit

To provide an interactive way to test the model, we will launch a Streamlit application. The following cell will import the necessary function and launch the app, providing a public URL for you to access.

In [ ]:
!pip install streamlit pyngrok nest_asyncio websockets > /dev/null

import sys
import os

# Add the parent directory of 'utils' to the path to find the inference_client
sys.path.append(os.path.abspath('../..'))
sys.path.append(os.path.abspath('../utils/streamlit_app'))


from utils import run_streamlit_direct

# Define the block_id and grpc_server_address
BLOCK_ID = "gemma3-27b-block2"
GRPC_SERVER_ADDRESS = "CLUSTER1MASTER:31500"
streamlit_url = run_streamlit_direct(BLOCK_ID, GRPC_SERVER_ADDRESS,port=8501)
print(f"Streamlit App URL: {streamlit_url}")

You can now open the URL above in your browser to interact with the model. Once you are done, you can proceed to the cleanup steps.

## 6. Cleaning Up

After you are finished with the block, it is important to deallocate it to free up resources. You can also deregister the asset if you no longer need it.


### Deallocate the Block
This command will stop the running block and release all associated resources. 

`K8s dashboard` available at https://CLUSTER1MASTER:32319/#/login

In [18]:
#Show the Block in K8s first and then show after deletion! 
!curl -X POST http://MANAGEMENTMASTER:30600/controller/removeBlock/gcp-cluster-2 -H "Content-Type: application/json" -d '{"block_id": "gemma3-27b-block2"}' | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   100  100    67  100    33     12      6  0:00:05  0:00:05 --:--:--    16
{
   "data" : {
      "data" : "Action performed",
      "success" : true
   },
   "success" : true
}


### Deregister the Asset
If you no longer need the asset in the registry, you can remove it.

In [19]:
!curl -X POST http://MANAGEMENTMASTER:30112/api/unregisterComponent -H "Content-Type: application/json" -d '{"uri": "model.gemma3-27b:1.0.0-stable"}' | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   104  100    64  100    40   7246   4528 --:--:-- --:--:-- --:--:-- 13000
{
   "error" : false,
   "payload" : {
      "acknowledged" : true,
      "deletedCount" : 1
   }
}


## More models sample notebooks can be found [here](https://github.com/OpenCyberspace/AIOS_AI_Blueprints/tree/main/video_tutorial_series/02_more_models_llama_cpp)